In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Layer, InputSpec
from tensorflow.keras.metrics import binary_accuracy
from tensorflow.keras.layers import Input, Dense, Concatenate, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

def binary_classification_loss(concat_true, concat_pred):
    t_true = concat_true[:, 1]
    t_pred = concat_pred[:, 2]
    t_pred = (t_pred + 0.001) / 1.002
    losst = tf.reduce_sum(K.binary_crossentropy(t_true, t_pred))

    return losst


def regression_loss(concat_true, concat_pred):
    y_true = concat_true[:, 0]
    t_true = concat_true[:, 1]

    y0_pred = concat_pred[:, 0]
    y1_pred = concat_pred[:, 1]

    loss0 = tf.reduce_sum((1. - t_true) * tf.square(y_true - y0_pred))
    loss1 = tf.reduce_sum(t_true * tf.square(y_true - y1_pred))

    return loss0 + loss1


def ned_loss(concat_true, concat_pred):
    t_true = concat_true[:, 1]

    t_pred = concat_pred[:, 1]
    return tf.reduce_sum(K.binary_crossentropy(t_true, t_pred))


def dead_loss(concat_true, concat_pred):
    return regression_loss(concat_true, concat_pred)


def dragonnet_loss_binarycross(concat_true, concat_pred):
    return regression_loss(concat_true, concat_pred) + binary_classification_loss(concat_true, concat_pred)


def treatment_accuracy(concat_true, concat_pred):
    t_true = concat_true[:, 1]
    t_pred = concat_pred[:, 2]
    return binary_accuracy(t_true, t_pred)



def track_epsilon(concat_true, concat_pred):
    epsilons = concat_pred[:, 3]
    return tf.abs(tf.reduce_mean(epsilons))


class EpsilonLayer(Layer):

    def __init__(self):
        super(EpsilonLayer, self).__init__()

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.epsilon = self.add_weight(name='epsilon',
                                       shape=[1, 1],
                                       initializer='RandomNormal',
                                       #  initializer='ones',
                                       trainable=True)
        super(EpsilonLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, **kwargs):
        # import ipdb; ipdb.set_trace()
        return self.epsilon * tf.ones_like(inputs)[:, 0:1]


def make_tarreg_loss(ratio=1., dragonnet_loss=dragonnet_loss_binarycross):
    def tarreg_ATE_unbounded_domain_loss(concat_true, concat_pred):
        vanilla_loss = dragonnet_loss(concat_true, concat_pred)

        y_true = concat_true[:, 0]
        t_true = concat_true[:, 1]

        y0_pred = concat_pred[:, 0]
        y1_pred = concat_pred[:, 1]
        t_pred = concat_pred[:, 2]

        epsilons = concat_pred[:, 3]
        t_pred = (t_pred + 0.01) / 1.02
        # t_pred = tf.clip_by_value(t_pred,0.01, 0.99,name='t_pred')

        y_pred = t_true * y1_pred + (1 - t_true) * y0_pred

        h = t_true / t_pred - (1 - t_true) / (1 - t_pred)

        y_pert = y_pred + epsilons * h
        targeted_regularization = tf.reduce_sum(tf.square(y_true - y_pert))

        # final
        loss = vanilla_loss + ratio * targeted_regularization
        return loss

    return tarreg_ATE_unbounded_domain_loss


def make_dragonnet(input_dim, reg_l2):
    """
    Neural net predictive model. The dragon has three heads.
    :param input_dim:
    :param reg:
    :return:
    """
    t_l1 = 0.
    t_l2 = reg_l2
    inputs = Input(shape=(input_dim,), name='input')

    # representation
    x = Dense(units=200, activation='elu', kernel_initializer='RandomNormal')(inputs)
    x = Dense(units=200, activation='elu', kernel_initializer='RandomNormal')(x)
    x = Dense(units=200, activation='elu', kernel_initializer='RandomNormal')(x)


    t_predictions = Dense(units=1, activation='sigmoid')(x)

    # HYPOTHESIS
    y0_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2))(x)
    y1_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2))(x)

    # second layer
    y0_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2))(y0_hidden)
    y1_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2))(y1_hidden)

    # third
    y0_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y0_predictions')(
        y0_hidden)
    y1_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y1_predictions')(
        y1_hidden)

    dl = EpsilonLayer()
    epsilons = dl(t_predictions, name='epsilon')
    # logging.info(epsilons)
    concat_pred = Concatenate(1)([y0_predictions, y1_predictions, t_predictions, epsilons])
    model = Model(inputs=inputs, outputs=concat_pred)

    return model


def make_tarnet(input_dim, reg_l2):
    """
    Neural net predictive model. The dragon has three heads.
    :param input_dim:
    :param reg:
    :return:
    """

    inputs = Input(shape=(input_dim,), name='input')

    # representation
    x = Dense(units=200, activation='elu', kernel_initializer='RandomNormal')(inputs)
    x = Dense(units=200, activation='elu', kernel_initializer='RandomNormal')(x)
    x = Dense(units=200, activation='elu', kernel_initializer='RandomNormal')(x)

    t_predictions = Dense(units=1, activation='sigmoid')(inputs)

    # HYPOTHESIS
    y0_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2))(x)
    y1_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2))(x)

    # second layer
    y0_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2))(y0_hidden)
    y1_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2))(y1_hidden)

    # third
    y0_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y0_predictions')(
        y0_hidden)
    y1_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y1_predictions')(
        y1_hidden)

    dl = EpsilonLayer()
    epsilons = dl(t_predictions, name='epsilon')
    # logging.info(epsilons)
    concat_pred = Concatenate(1)([y0_predictions, y1_predictions, t_predictions, epsilons])
    model = Model(inputs=inputs, outputs=concat_pred)

    return model


def make_ned(input_dim, reg_l2=0.01):
    """
    Neural net predictive model. The dragon has three heads.
    :param input_dim:
    :param reg:
    :return:
    """

    inputs = Input(shape=(input_dim,), name='input')

    # representation
    x = Dense(units=200, activation='elu', kernel_initializer='RandomNormal', name='ned_hidden1')(inputs)
    x = Dense(units=200, activation='elu', kernel_initializer='RandomNormal', name='ned_hidden2')(x)
    x = Dense(units=200, activation='elu', kernel_initializer='RandomNormal', name='ned_hidden3')(x)

    t_predictions = Dense(units=1, activation='sigmoid', name='ned_t_activation')(x)
    y_predictions = Dense(units=1, activation=None, name='ned_y_prediction')(x)

    concat_pred = Concatenate(1)([y_predictions, t_predictions])

    model = Model(inputs=inputs, outputs=concat_pred)
    return model


def post_cut(nednet, input_dim, reg_l2=0.01):
    for layer in nednet.layers:
        layer.trainable = False
    nednet.layers.pop()
    nednet.layers.pop()
    nednet.layers.pop()

    frozen = nednet

    x = frozen.layers[-1].output
    frozen.layers[-1].outbound_nodes = []
    input = frozen.input

    y0_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2), name='post_cut_y0_1')(x)
    y1_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2), name='post_cut_y1_1')(x)

    # second layer
    y0_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2), name='post_cut_y0_2')(
        y0_hidden)
    y1_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2), name='post_cut_y1_2')(
        y1_hidden)

    # third
    y0_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y0_predictions')(
        y0_hidden)
    y1_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y1_predictions')(
        y1_hidden)

    concat_pred = Concatenate(1)([y0_predictions, y1_predictions])

    model = Model(inputs=input, outputs=concat_pred)
    return 

In [33]:
import os
import glob
import argparse
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau, TerminateOnNaN


def _split_output(yt_hat, t, y, y_scaler,):
    q_t0 = y_scaler.inverse_transform(yt_hat[:, 0][:,None].copy())
    q_t1 = y_scaler.inverse_transform(yt_hat[:, 1][:,None].copy())
    g = yt_hat[:, 2].copy()

    if yt_hat.shape[1] == 4:
        eps = yt_hat[:, 3][0]
    else:
        eps = np.zeros_like(yt_hat[:, 2])

    y = y_scaler.inverse_transform(y.copy())
    var = "average propensity for treated: {} and untreated: {}".format(g[t.squeeze() == 1.].mean(),
                                                                        g[t.squeeze() == 0.].mean())
    print(var)

    return {'q_t0': q_t0, 'q_t1': q_t1, 'g': g, 't': t, 'y': y}


def train_and_predict_dragons(train_data, train_label, yf, test_data, t_test,yf_test,
                              targeted_regularization=True, 
                              knob_loss=dragonnet_loss_binarycross, 
                              ratio=1., dragon='', 
                              val_split=0.2, batch_size=64):
    verbose = 0
    y_scaler = StandardScaler().fit(yf)
    y = y_scaler.transform(yf)    
    train_outputs = []
    test_outputs = []

    if dragon == 'tarnet':
        dragonnet = make_tarnet(train_data.shape[1], 0.01)

    elif dragon == 'dragonnet':
        print("I am here making dragonnet")
        dragonnet = make_dragonnet(train_data.shape[1], 0.01)

    metrics = [regression_loss, 
               binary_classification_loss, 
               treatment_accuracy, 
               track_epsilon]

    if targeted_regularization:
        loss = make_tarreg_loss(ratio=ratio, dragonnet_loss=knob_loss)
    else:
        loss = knob_loss

    # for reporducing the IHDP experimemt

    i = 0
    tf.random.set_seed(i)
    np.random.seed(i)


    yt_train = np.concatenate([y, train_label[:,0][:, None]], 1)

    import time;
    start_time = time.time()

    dragonnet.compile(
        optimizer=Adam(learning_rate=1e-3),
        loss=loss, metrics=metrics)

    adam_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=2, min_delta=0.),
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=1e-8, cooldown=0, min_lr=0)

    ]

    dragonnet.fit(train_data, yt_train, callbacks=adam_callbacks,
                  validation_split=val_split,
                  epochs=100,
                  batch_size=batch_size, verbose=verbose)

    sgd_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=40, min_delta=0.),
        ReduceLROnPlateau(monitor='loss', 
                          factor=0.5, 
                          patience=5, 
                          verbose=verbose, mode='auto',
                          min_delta=0., cooldown=0, min_lr=0)
    ]

    sgd_lr = 1e-5
    momentum = 0.9
    dragonnet.compile(optimizer=SGD(learning_rate=sgd_lr, 
                                    momentum=momentum, 
                                    nesterov=True), loss=loss,
                      metrics=metrics)
    dragonnet.fit(train_data, yt_train, callbacks=sgd_callbacks,
                  validation_split=val_split,
                  epochs=300,
                  batch_size=batch_size, verbose=verbose)

    elapsed_time = time.time() - start_time
    print("***************************** elapsed_time is: ", elapsed_time)

    yt_hat_test = dragonnet.predict(test_data)
    yt_hat_train = dragonnet.predict(train_data)
    test_outputs += [_split_output(yt_hat_test, t_test, yf_test, y_scaler)]
    train_outputs += [_split_output(yt_hat_train, train_label[:,0], yf, y_scaler)]
    K.clear_session()

    return test_outputs, train_outputs



In [47]:
import pandas as pd
import numpy as np
from tqdm import tqdm,trange
path = 'csv_files/' #path of IHDP files, downloaded from https://www.fredjo.com/
def processed_data(i):
    train_data = pd.read_csv(path+'1000_train'+str(i)+'.csv').values
    train_data[:,13] = train_data[:,13]-1 #processing according to CEVAE
    test_data = pd.read_csv(path+'1000_test'+str(i)+'.csv').values
    test_data[:,13] = test_data[:,13]-1 #processing according to CEVAE
    train_label = np.zeros((len(train_data),3))
    t= pd.read_csv(path+'1000_train_t'+str(i)+'.csv').values.flatten()
    train_label[:,0] = t
    yf = pd.read_csv(path+'1000_train_yf'+str(i)+'.csv').values.flatten()
    ycf = pd.read_csv(path+'1000_train_ycf'+str(i)+'.csv').values.flatten()
    train_label[:,1] = np.where(t==1,yf, ycf) #treatment effect
    train_label[:,2] = np.where(t==0,yf, ycf) #non-treatment effect
    train_ite = np.zeros((len(train_data),1))
    train_ite = np.where(t==1,yf-ycf, ycf-yf)
    ite_test = pd.read_csv(path+'1000_test_ite'+str(i)+'.csv').values
    t_test= pd.read_csv(path+'1000_test_t'+str(i)+'.csv').values.flatten()
    yf_test = pd.read_csv(path+'1000_test_yf'+str(i)+'.csv').values.flatten()
    return train_data,test_data,train_label,\
            yf[:, None],ycf[:, None],\
            train_ite,ite_test,t_test,yf_test[:, None]
test_losses_regularized=[]
test_losses_unregularized=[]
for i in trange(1000, position=0, desc="replication", leave=True, colour='black',):
    train_data,test_data,train_label,train_yf, train_ycf, ite_train, ite_test,t_test,yf_test = processed_data(i)   
    knob_loss=dragonnet_loss_binarycross
    ratio=1.
    for is_targeted_regularization in [True, False]:
        print("Is targeted regularization: {}".format(is_targeted_regularization))
        test_outputs, train_outputs= train_and_predict_dragons(
            train_data, train_label, train_yf, test_data,t_test,yf_test,dragon='dragonnet', 
            targeted_regularization=is_targeted_regularization,
            knob_loss=knob_loss, ratio=ratio, 
            val_split=0.2, batch_size=64)
        ite_test_preds = test_outputs[0]['q_t1'] - test_outputs[0]['q_t0']
        if is_targeted_regularization:
            test_losses_regularized.append(np.mean((ite_test-ite_test_preds)**2)**0.5)
        else:    
            test_losses_unregularized.append(np.mean((ite_test-ite_test_preds)**2)**0.5)

replication:   0%|                                     | 0/1000 [00:00<?, ?it/s]

Is targeted regularization: True
I am here making dragonnet
***************************** elapsed_time is:  12.942372798919678
21/21 [==============================] - 0s 1ms/step
average propensity for treated: 0.20356741547584534 and untreated: 0.15415646135807037
average propensity for treated: 0.23915964365005493 and untreated: 0.16960518062114716
Is targeted regularization: False
I am here making dragonnet
***************************** elapsed_time is:  11.799004077911377
21/21 [==============================] - 0s 1ms/step
average propensity for treated: 0.20097461342811584 and untreated: 0.14850737154483795
average propensity for treated: 0.24930088222026825 and untreated: 0.16596658527851105


replication:   0%|                           | 1/1000 [00:25<7:04:11, 25.48s/it]

Is targeted regularization: True
I am here making dragonnet
***************************** elapsed_time is:  10.590492486953735
21/21 [==============================] - 0s 999us/step
average propensity for treated: 0.24701261520385742 and untreated: 0.18703119456768036
average propensity for treated: 0.24805322289466858 and untreated: 0.18860778212547302
Is targeted regularization: False
I am here making dragonnet
***************************** elapsed_time is:  8.467555046081543
21/21 [==============================] - 0s 994us/step
average propensity for treated: 0.24856846034526825 and untreated: 0.17245998978614807
average propensity for treated: 0.2817503809928894 and untreated: 0.17936301231384277


replication:   0%|                           | 2/1000 [00:45<6:07:04, 22.07s/it]

...

average propensity for treated: 0.26625150442123413 and untreated: 0.14925441145896912
Is targeted regularization: True
I am here making dragonnet
***************************** elapsed_time is:  11.497830867767334
21/21 [==============================] - 0s 1ms/step
average propensity for treated: 0.2333599328994751 and untreated: 0.18817421793937683
average propensity for treated: 0.25791624188423157 and untreated: 0.16919319331645966
Is targeted regularization: False
I am here making dragonnet
***************************** elapsed_time is:  7.820310592651367
21/21 [==============================] - 0s 868us/step
average propensity for treated: 0.23445741832256317 and untreated: 0.18599292635917664
average propensity for treated: 0.27940377593040466 and untreated: 0.16331961750984192


replication: 100%|████████████████████████▉| 999/1000 [6:51:56<00:23, 23.61s/it]

Is targeted regularization: True
I am here making dragonnet
***************************** elapsed_time is:  13.433315515518188
21/21 [==============================] - 0s 1ms/step
average propensity for treated: 0.20506708323955536 and untreated: 0.18821774423122406
average propensity for treated: 0.2734644412994385 and untreated: 0.15893083810806274
Is targeted regularization: False
I am here making dragonnet
***************************** elapsed_time is:  12.549269199371338
21/21 [==============================] - 0s 1ms/step


replication: 100%|████████████████████████| 1000/1000 [6:52:23<00:00, 24.74s/it]

average propensity for treated: 0.2166612297296524 and untreated: 0.19129057228565216
average propensity for treated: 0.28020811080932617 and untreated: 0.16062134504318237


In [48]:
#print the results using regularized dragonnet
from scipy.stats import sem
results = np.array(test_losses_regularized)
print(
    '\n The average rpehe of test data are: ',
    np.mean(results),
    'The standard error of epehe in test data are: ',
    sem(results))


 The average rpehe of test data are:  1.392944381711027 The standard error of epehe in test data are:  0.05188035812793893


In [49]:
#print the results using unregularized dragonnet
from scipy.stats import sem
results = np.array(test_losses_unregularized)
print(
    '\n The average rpehe of test data are: ',
    np.mean(results),
    'The standard error of epehe in test data are: ',
    sem(results))


 The average rpehe of test data are:  1.5450654508126833 The standard error of epehe in test data are:  0.061287040292049066


In [50]:
import pandas as pd
import numpy as np
from tqdm import tqdm,trange
path = 'csv_files/' #path of IHDP files, downloaded from https://www.fredjo.com/
def processed_data(i):
    train_data = pd.read_csv(path+'1000_train'+str(i)+'.csv').values
    train_data[:,13] = train_data[:,13]-1 #processing according to CEVAE
    test_data = pd.read_csv(path+'1000_test'+str(i)+'.csv').values
    test_data[:,13] = test_data[:,13]-1 #processing according to CEVAE
    train_label = np.zeros((len(train_data),3))
    t= pd.read_csv(path+'1000_train_t'+str(i)+'.csv').values.flatten()
    train_label[:,0] = t
    yf = pd.read_csv(path+'1000_train_yf'+str(i)+'.csv').values.flatten()
    ycf = pd.read_csv(path+'1000_train_ycf'+str(i)+'.csv').values.flatten()
    train_label[:,1] = np.where(t==1,yf, ycf) #treatment effect
    train_label[:,2] = np.where(t==0,yf, ycf) #non-treatment effect
    train_ite = np.zeros((len(train_data),1))
    train_ite = np.where(t==1,yf-ycf, ycf-yf)
    ite_test = pd.read_csv(path+'1000_test_ite'+str(i)+'.csv').values
    t_test= pd.read_csv(path+'1000_test_t'+str(i)+'.csv').values.flatten()
    yf_test = pd.read_csv(path+'1000_test_yf'+str(i)+'.csv').values.flatten()
    return train_data,test_data,train_label,\
            yf[:, None],ycf[:, None],\
            train_ite,ite_test,t_test,yf_test[:, None]
tarnet_losses_regularized=[]
tarnet_losses_unregularized=[]
for i in trange(1000, position=0, desc="replication", leave=True, colour='black',):
    train_data,test_data,train_label,train_yf, train_ycf, ite_train, ite_test,t_test,yf_test = processed_data(i)   
    knob_loss=dragonnet_loss_binarycross
    ratio=1.
    for is_targeted_regularization in [True, False]:
        print("Is targeted regularization: {}".format(is_targeted_regularization))
        test_outputs, train_outputs= train_and_predict_dragons(
            train_data, train_label, train_yf, test_data,t_test,yf_test,dragon='tarnet', 
            targeted_regularization=is_targeted_regularization,
            knob_loss=knob_loss, ratio=ratio, 
            val_split=0.2, batch_size=64)
        ite_test_preds = test_outputs[0]['q_t1'] - test_outputs[0]['q_t0']
        if is_targeted_regularization:
            tarnet_losses_regularized.append(np.mean((ite_test-ite_test_preds)**2)**0.5)
        else:    
            tarnet_losses_unregularized.append(np.mean((ite_test-ite_test_preds)**2)**0.5)

replication:   0%|                                     | 0/1000 [00:00<?, ?it/s]

Is targeted regularization: True


2023-01-21 22:42:12.074446: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-01-21 22:42:15.001830: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


***************************** elapsed_time is:  12.544079780578613
21/21 [==============================] - 0s 1ms/step
average propensity for treated: 0.19513897597789764 and untreated: 0.186791330575943
average propensity for treated: 0.20887792110443115 and untreated: 0.19444885849952698
Is targeted regularization: False
***************************** elapsed_time is:  17.754586935043335
21/21 [==============================] - 0s 981us/step


replication:   0%|                           | 1/1000 [00:31<8:37:33, 31.09s/it]

...

replication: 100%|████████████████████████▉| 998/1000 [9:08:21<00:59, 29.56s/it]

average propensity for treated: 0.2026335597038269 and untreated: 0.18832005560398102
average propensity for treated: 0.233672097325325 and untreated: 0.19028235971927643
Is targeted regularization: True
***************************** elapsed_time is:  13.427828073501587
21/21 [==============================] - 0s 1ms/step
average propensity for treated: 0.28905656933784485 and untreated: 0.21864032745361328
average propensity for treated: 0.2521488666534424 and untreated: 0.21912053227424622
Is targeted regularization: False
***************************** elapsed_time is:  17.630648851394653
21/21 [==============================] - 0s 979us/step


replication: 100%|████████████████████████▉| 999/1000 [9:08:53<00:30, 30.21s/it]

average propensity for treated: 0.22733040153980255 and untreated: 0.22708457708358765
average propensity for treated: 0.22779898345470428 and untreated: 0.21246904134750366
Is targeted regularization: True
***************************** elapsed_time is:  16.212899208068848
21/21 [==============================] - 0s 1ms/step
average propensity for treated: 0.18219423294067383 and untreated: 0.19287467002868652
average propensity for treated: 0.21125969290733337 and untreated: 0.1990625262260437
Is targeted regularization: False
***************************** elapsed_time is:  17.608607292175293
21/21 [==============================] - 0s 1ms/step
average propensity for treated: 0.2088172882795334 and untreated: 0.19582393765449524
average propensity for treated: 0.22520652413368225 and untreated: 0.192121222615242


replication: 100%|████████████████████████| 1000/1000 [9:09:27<00:00, 32.97s/it]


In [51]:
#print the results using regularized dragonnet
from scipy.stats import sem
results = np.array(tarnet_losses_regularized)
print(
    '\n The average rpehe of test data are: ',
    np.mean(results),
    'The standard error of epehe in test data are: ',
    sem(results))


 The average rpehe of test data are:  1.2416321274383262 The standard error of epehe in test data are:  0.04038793869014789


In [52]:
#print the results using regularized dragonnet
from scipy.stats import sem
results = np.array(tarnet_losses_unregularized)
print(
    '\n The average rpehe of test data are: ',
    np.mean(results),
    'The standard error of epehe in test data are: ',
    sem(results))


 The average rpehe of test data are:  1.2837404014262328 The standard error of epehe in test data are:  0.04124622916256353
